In [1]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

import sys
reload(sys)
sys.setdefaultencoding('ISO-8859-1')

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/Users/xiangyusi/.matplotlib/matplotlibrc", line #2
  (fname, cnt))


### 1. Load the training dataset

In [2]:
pd.set_option('display.max_column', None)
training_set = pd.read_csv('train.csv')

# we want to separate the txt messages into two groups: ham(1) and spam(0)
ham_or_spam = training_set['label']
training_set_target = [] # same as the label in Ziyi's version
for i in range(len(ham_or_spam)):
    if ham_or_spam[i] == "ham":
        training_set_target.append(1)
    else:
        training_set_target.append(0)
# y is the array version
y = training_set_target
training_set_target = np.transpose(np.matrix(training_set_target))
training_set_content = training_set['sms'].as_matrix()

In [3]:
''' Load the stopwords '''
stop_words = stopwords.words('english')
stop_words[:10]

[u'i',
 u'me',
 u'my',
 u'myself',
 u'we',
 u'our',
 u'ours',
 u'ourselves',
 u'you',
 u'your']

### 2. Tokenizing

In [4]:
def tokenization(text):
    tokens=[]
    for word in nltk.word_tokenize(text):
        if re.search('[a-zA-Z]', word) and word.lower() not in stop_words:
            tokens.append(word.lower())
    return tokens

training_set_corpus_tokenized = []
for i in training_set_content:
    training_set_corpus_tokenized.append(' '.join(tokenization(i)))
    
training_set_corpus_tokenized[:10]

['hey pay salary de lt gt',
 'happen dear silent tensed',
 'want please inside outside bedroom',
 'wonder got online love gone net cafe get phone recharged friends net think boytoy',
 'dad gon na call gets work ask crazy questions',
 'good afternoon love goes day hope maybe got leads job think boytoy send passionate kiss across sea',
 "u repeat e instructions wat 's e road name ur house",
 'yo dude guess got arrested day',
 'nokia tone ur mob every week txt nok 1st tone free get txtin tell ur friends 150p/tone reply hl 4info',
 'hello damn christmas thing think decided keep mp3 doesnt work']

### 3. Applying TF-IDF

In [5]:
vectorizer = CountVectorizer(decode_error = 'ignore')
transformer = TfidfTransformer(norm = 'l2', use_idf = True)
tfidf_matrix = transformer.fit_transform(vectorizer.fit_transform(training_set_corpus_tokenized))

X = tfidf_matrix.toarray()

### 4. Define functions for logistic regression
Because do not want to regularize the bias term (in order to shift the plot) ==> separate the gradient of the first dimension apart from the others.

In [6]:
def add_bias_term(X):
    X = np.insert(X, [0], [1], axis=1)
    return X

def sigmoid(z):
    sigma = 1 / (1 + np.exp(-z))
    return sigma

def loss_function(X, y, w, lmd):
    # X is the training set, 3000 x 3000, w is the weight vector, 3000 x 1
    # y is the target, 3000 x 1, lmd is the hyper parameter for regularization
    m = np.shape(y)[0]
    y = np.transpose(y)
    hypo = sigmoid(np.dot(X, w))
    # avoid the case of log(0)
    for i in range(len(hypo)):
        if hypo[i,0] == 1:
            hypo[i,0] = 0.99999
        elif hypo[i,0] < 0.00001:
            hypo[i,0] = 0.00001
    loss = (1.0 / m) * (-np.dot(y, np.log(hypo)) - np.dot((1 - y), np.log(1 - hypo))) + lmd * (1.0 / m) * np.dot(np.transpose(w), w)
    return loss

def gradient_descent(X, y, w_, eta, lmd):
    m = np.shape(y)[0]
    hypo = sigmoid(np.dot(X, w_))
    loss_history = []
    # store every previous loss value to monitor whether the gradient descent is improving
    # this mechanism also helps to avoid 'overshooting'
    prev_loss = 9999.0
    for i in range(1, 1001):
        # below is the general form of gradient descent:
        # w_ = w_ - (1.0 / m) * eta * (i ** (-0.9)) * np.dot(np.transpose(X), hypo - y) + lmd * w_
        w_[0,:] = w_[0,:] - (1.0 / m) * eta * (i ** (-0.9)) * np.dot(np.transpose(X)[0,:], hypo - y)
        w_[1:,:] = w_[1:,:] - (1.0 / m) * eta * (i ** (-0.9)) * np.dot(np.transpose(X)[1:,:], hypo - y) + lmd * 1.0 / m * w_[1:,:]
        crt_loss = loss_function(X, y, w_, lmd)[0,0]
        if crt_loss > prev_loss:
            break
        loss_history.append(crt_loss)
        prev_loss = crt_loss
    print loss_history
    return w_

def predict(X, y, w):
    predict_hypo = sigmoid(np.dot(X, w))
    p = predict_hypo
    for i in range(np.shape(y)[0]):
        if predict_hypo[i,0] >= 0.5:
            p[i,0] = 1
        else:
            p[i,0] = 0
    return p

def compute_accuracy(X, y, w):
    result = predict(X, y, w)
    wrong_answer = 0.0
    for i in range(np.shape(y)[0]):
        if result[i,0] != y[i,0]:
            wrong_answer = wrong_answer + 1
    accuracy = 1 - wrong_answer / np.shape(y)[0]
    return accuracy

Run the model. <br>
Originally set the learning rate to be 1, and it continues to drop down. Also, because don't have bias term, set regrlarizer to 0.

In [7]:
w = np.transpose(np.matrix(np.zeros(np.shape(X)[1])))
w = gradient_descent(X, training_set_target, w, 1, 0)

w

matrix([[-0.00055179],
        [-0.00048387],
        [-0.00048387],
        ...,
        [ 0.00066489],
        [ 0.00050107],
        [ 0.00028816]])

Compute the accuracy of training set to evaluate the model and get the accuracy without bias term.

### 5. Load the test data and compute accuracy
Compute accuracy for test data

In [8]:
test_set = pd.read_csv('test.csv')

ham_or_spam = test_set['label']
test_set_target = []
for i in range(len(ham_or_spam)):
    if ham_or_spam[i] == "ham":
        test_set_target.append(1)
    else:
        test_set_target.append(0)
y_test = test_set_target
test_set_target = np.transpose(np.matrix(test_set_target))

test_set_content = test_set['sms'].as_matrix()
test_set_corpus_tokenized = []
for i in test_set_content:
    test_set_corpus_tokenized.append(' '.join(tokenization(i)))

tfidf_test = transformer.transform(vectorizer.transform(test_set_corpus_tokenized))
X_test = tfidf_test.toarray()

accuracy_test = compute_accuracy(X_test, test_set_target, w)

accuracy_test

0.9447900466562986

### 6. Add bias term

In [9]:
X = add_bias_term(X)
# initialize w to be a matrix of dimension 3000 x 1 with all elements to be 0
w = np.transpose(np.matrix(np.zeros(np.shape(X)[1])))
w = gradient_descent(X, training_set_target, w, 1, 0.05)

w

matrix([[ 1.83550768e+00],
        [-2.66734942e-04],
        [-2.33901988e-04],
        ...,
        [ 3.21408940e-04],
        [ 2.42217571e-04],
        [ 1.39297406e-04]])

Compute accuracy for training set with bias term

In [10]:
accuracy = compute_accuracy(X, training_set_target, w)

accuracy

0.861

Compute accuracy for test set with bias term

In [11]:
X_test = add_bias_term(X_test)
accuracy_test = compute_accuracy(X_test, test_set_target, w)

accuracy_test

0.8716951788491446

### 7. Cross validation
In practice, 10-fold cross validation works extremely slowly. Also, find out that in both models (with or without bias term), accuracies do not change obviously when changing hyperparameter within a small range (around 0 - 0.1). However, while the hyperparameter exceeds this range, the iteration of gradient descent pauses early in order to avoid 'overshooting'.

In [ ]:
# return the lmd that maximize the accuracy
def cross_validation(X, w, y, eta):
    # divide the dataset into 10 folds
    kf = KFold(n_splits = 10)
    training_cost_avg = 0
    test_cost_avg = 0
    accuracy_history = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
    lmd = np.linspace(0, 1, num=100)
    for i in range(len(lmd)):
        w = np.transpose(np.matrix(np.zeros(np.shape(X_train)[1])))
        w = gradient_descent(X_train, y_train, w, eta, lmd[i])
        accuracy = compute_accuracy(X_test, y_test, w)
        accuracy_history.append(accuracy)
    # find the index of this best lmd
    best_lmd_index = np.argmax(accuracy_history)
    # find it in the lmd array
    lmd_best = lmd[best_lmd_index]
    return lmd_best